In [ ]:
#@title # 👇 **下載 (可以執行多次)**

%cd /content
link = "" #@param{type:"string"}
from_direct_link = False #@param{type:"boolean"}
from_torrent = True #@param{type:"boolean"}
from_gigafile = False #@param{type:"boolean"}

!mkdir -p /content/DL
%cd /content/DL

if from_torrent or from_direct_link or from_gigafile:
  !apt install -y -qq aria2

if from_torrent:
  !aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c --summary-interval=10 -c -x 10 -k 1M -s 10 "{link}"

if from_gigafile:
  !pip install gigafile
  !gfile download "{link}"

print("\nDone!")

In [ ]:
#@title # 👇 **gofile下載 (限單檔)**
%cd /content
!git clone https://github.com/rushiaismywaifu/megaloader.git
%cd /content/megaloader
!mkdir -p /content/DL
!python -m pip install -r requirements.txt
from plugins import GoFile

output = "/content/DL"
url = "https://gofile.io/d/oUhtUC" #@param{type:"string"}
api = GoFile(url)
for u in api.export():
    api.download_file(u, output)

%cd /content/DL
print("\nDone!")

In [ ]:
#@title # 👇 **上傳至 Google Drive (資料夾位置記得加 "/" )**
from google.colab import drive
import os
if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
%cd /content/DL
output_path = "ColabDownloads/" #@param {type:"string"}
!rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}
print("\nDone!")

In [ ]:
#@title # 👇 **上傳至Gofile (單檔)**
from google.colab import output
import requests


def response_handler(response):
  if response["status"] == "ok":
    return response["data"]

  elif "error-" in response["status"]:
    error = response["status"].split("-")[1]
    raise Exception(error)


def getServer():
  getServer_response = requests.get(url="https://api.gofile.io/getServer").json()
  return response_handler(getServer_response)


server = getServer()["server"]
url = f"https://{server}.gofile.io/uploadFile"
!fallocate -l 1 Create
data = response_handler(requests.post(url=url, files={"upload_file": open("Create", "rb")}).json())

name = "someFile.txt" #@param {type:"string"}
!curl -F "file=@{name}" -F token={data['guestToken']} -F folderId={data['parentFolder']} {url}
!curl 'https://api.gofile.io/deleteContent' -X DELETE --data-raw 'contentsId={data["fileId"]}&token={data["guestToken"]}'
!rm Create
output.clear()

print("下載網址： ", data['downloadPage'])

In [ ]:
#@title # 👇 **上傳至Gofile (多檔)**
from google.colab import output
import requests, os


def response_handler(response):
  if response["status"] == "ok":
    return response["data"]

  elif "error-" in response["status"]:
    error = response["status"].split("-")[1]
    raise Exception(error)


def getServer():
  getServer_response = requests.get(url="https://api.gofile.io/getServer").json()
  return response_handler(getServer_response)


server = getServer()["server"]
url = f"https://{server}.gofile.io/uploadFile"
!fallocate -l 1 Create
data = response_handler(requests.post(url=url, files={"upload_file": open("Create", "rb")}).json())
!rm Create

allpath = "/content/DL"
for root, dirs, files in os.walk(allpath):
  for f in files:
    fullpath = os.path.join(root, f)
    server = getServer()["server"]
    url = f"https://{server}.gofile.io/uploadFile"
    !curl -F "file=@{fullpath}" -F token={data['guestToken']} -F folderId={data['parentFolder']} {url}
!curl 'https://api.gofile.io/deleteContent' -X DELETE --data-raw 'contentsId={data["fileId"]}&token={data["guestToken"]}'
output.clear()

print("下載網址： ", data['downloadPage'])

In [ ]:
#@title # 👇 **上傳至Pixeldrain (限20GB以下)**
from google.colab import output
filename = "someFile.txt" #@param {type:"string"}
a = !curl -T "{filename}" https://pixeldrain.com/api/file/
# output.clear()
print("下載網址： https://pixeldrain.com/u/", a[0][7:-2], sep='')

In [ ]:
#@title # 👇 **上傳至Gigafile**
!pip install gigafile
somefile = "someFile.txt" #@param {type:"string"}
!gfile -p -n 24 upload "{somefile}"

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")